In [2]:
!pip install deepface
!pip install streamlit
!pip install ultralytics
import streamlit as st
import numpy as np
import tensorflow as tf
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import SimpleRNN as OriginalSimpleRNN, Embedding
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import cv2
import tempfile
import os
from deepface import DeepFace
from ultralytics import YOLO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=ed9b083e25474052b438e3c7124fe40bae3e03965a2a4c0a3a4641f4b63c764d
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px
import plotly.graph_objs as go


In [4]:
# Load IMDb word index

def load_word_index():
    return imdb.get_word_index()

In [5]:
#Model
def load_sentiment_model():
    class CustomSimpleRNN(OriginalSimpleRNN):
        def __init__(self, *args, **kwargs):
            kwargs.pop('time_major', None)
            super().__init__(*args, **kwargs)
    return load_model('simple_rnn_imdb.h5', custom_objects={'SimpleRNN': CustomSimpleRNN})

In [6]:
#Preprocess
def preprocess_text(text, word_index):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    return sequence.pad_sequences([encoded_review], maxlen=500)
def predict_sentiment(text, model, word_index):
    preprocessed = preprocess_text(text, word_index)
    prediction = model.predict(preprocessed)
    sentiment = "Positive" if prediction[0][0] > 0.7 else "Negative"
    return sentiment, float(prediction[0][0])

In [7]:
# Loading GPT-2 model for generation

def load_text_generator():
    return pipeline("text-generation", model="gpt2")
def generate_explanation(text, generator):
    prompt = f"Review: {text}\nSentiment:"
    output = generator(prompt, max_length=100)[0]['generated_text']
    return output


In [8]:
# Word embedding demo
def embedding_demo(sentence, voc_size=10000, sent_length=8, dim=10):
    one_hot_repr = one_hot(sentence, voc_size)
    padded_seq = pad_sequences([one_hot_repr], maxlen=sent_length, padding='pre')
    model = tf.keras.Sequential()
    model.add(Embedding(voc_size, dim, input_length=sent_length))
    model.compile('adam', 'mse')
    embeddings = model.predict(padded_seq)
    return padded_seq, embeddings


In [9]:
# Load YOLOv8 model

def load_yolo_model():
    return YOLO("yolov8n.pt")

In [10]:
def analyze_image(image_file):
    tfile = tempfile.NamedTemporaryFile(delete=False)
    tfile.write(image_file.read())
    img_path = tfile.name

    yolo_model = load_yolo_model()
    yolo_results = yolo_model(img_path)[0]
    yolo_labels = [yolo_model.names[int(cls)] for cls in yolo_results.boxes.cls]

    try:
        face_result = DeepFace.analyze(img_path, actions=['emotion'], enforce_detection=False)[0]
        emotion = face_result['dominant_emotion']
    except:
        emotion = "Face not detected"

    return yolo_labels, emotion


In [11]:
def analyze_video(video_file):
    tfile = tempfile.NamedTemporaryFile(delete=False, suffix='.mp4')
    tfile.write(video_file.read())
    cap = cv2.VideoCapture(tfile.name)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_rate = max(frame_count // 10, 1)

    yolo_model = load_yolo_model()
    object_counts = {}
    emotions = []
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % sample_rate == 0:
            tmp_path = os.path.join(tempfile.gettempdir(), f"frame_{frame_idx}.jpg")
            cv2.imwrite(tmp_path, frame)
            yolo_results = yolo_model(tmp_path)[0]
            for cls in yolo_results.boxes.cls:
                label = yolo_model.names[int(cls)]
                object_counts[label] = object_counts.get(label, 0) + 1
            try:
                face_result = DeepFace.analyze(tmp_path, actions=['emotion'], enforce_detection=False)[0]
                emotions.append(face_result['dominant_emotion'])
            except:
                pass
        frame_idx += 1

    cap.release()
    dominant_objects = sorted(object_counts.items(), key=lambda x: x[1], reverse=True)[:5]
    dominant_emotion = max(set(emotions), key=emotions.count) if emotions else "None"
    return dominant_objects, dominant_emotion

In [12]:
# Fusion logic
def fuse_sentiment(text_sentiment, image_emotion):
    if text_sentiment == "Positive" and image_emotion in ["happy", "surprise"]:
        return "Strongly Positive"
    elif text_sentiment == "Negative" and image_emotion in ["angry", "sad"]:
        return "Strongly Negative"
    else:
        return "Mixed Sentiment"

In [13]:
def plot_attention_heatmap(attention_weights, tokens):
    fig, ax = plt.subplots(figsize=(10, 5))
    sns.heatmap(attention_weights, xticklabels=tokens, yticklabels=tokens, cmap='viridis', ax=ax)
    st.pyplot(fig)

In [14]:
def plot_cosine_similarity(query_embedding, docs_embeddings):
    sim_matrix = cosine_similarity([query_embedding], docs_embeddings)
    fig, ax = plt.subplots(figsize=(8, 2))
    sns.heatmap(sim_matrix, annot=True, cmap='coolwarm', cbar=True)
    st.pyplot(fig)


In [15]:
def pca_kmeans_plot(embeddings, n_clusters=4):
    pca = PCA(n_components=2)
    reduced = pca.fit_transform(embeddings)
    kmeans = KMeans(n_clusters=n_clusters)
    labels = kmeans.fit_predict(reduced)
    fig = px.scatter(x=reduced[:, 0], y=reduced[:, 1], color=labels.astype(str), title='PCA + KMeans Clustering')
    st.plotly_chart(fig)

In [16]:
def plot_3d_embeddings(embeddings):
    pca = PCA(n_components=3)
    reduced = pca.fit_transform(embeddings)
    fig = px.scatter_3d(x=reduced[:, 0], y=reduced[:, 1], z=reduced[:, 2], title='3D Embedding Scatter')
    st.plotly_chart(fig)

In [17]:
def show_clustermap(matrix, labels):
    sns.clustermap(matrix, cmap='vlag', xticklabels=labels, yticklabels=labels)
    st.pyplot()